In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from Models.Transformer import Transformer

2024-03-13 11:52:40.629322: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/jonathankonig/opt/anaconda3/envs/Tensorflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
tfds.load("scan/addprim_jump")

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 34.24 url/s]
Extraction completed...: 0 file [00:00, ? file/s]
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 15.41 url/s]


Dataset scan downloaded and prepared to /Users/jonathankonig/tensorflow_datasets/scan/addprim_jump/1.1.1. Subsequent calls will reuse this data.


{Split('train'): <_PrefetchDataset element_spec={'actions': TensorSpec(shape=(), dtype=tf.string, name=None), 'commands': TensorSpec(shape=(), dtype=tf.string, name=None)}>,
 Split('test'): <_PrefetchDataset element_spec={'actions': TensorSpec(shape=(), dtype=tf.string, name=None), 'commands': TensorSpec(shape=(), dtype=tf.string, name=None)}>}

In [ ]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1